In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os
from pathlib import Path

2025-07-14 11:17:03.614588: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 11:17:03.804763: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-14 11:17:03.808887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-07-14 11:17:03.808924: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [ ]:
SAMPLING_RATE = 9800 # Частота дискретизации 
WINDOW_SIZE = 4096 # Размер фрейма для дробления исходных данных   
STEP = 2048 # Шаг фрейма

NPERSEG = 512  # Размер фрейма для создания спектрограммы            
NOVERLAP = NPERSEG - 1 #коэф наложения фрейма

DPI = 100 # Количество точек на дюйм для изображения
IMG_HEIGHT = 257  # Высота изображения 
IMG_WIDTH = 3585 # Ширина изображения
CLASS_NAMES = ['defect', 'normal'] # Классы модели, важен порядок (смотри train.ipynb)

In [3]:
def create_spectrogram_from_chunk_normalized(chunk_data, fs):
    """
    Создает нормализованную спектрограмму из фрагмента данных в памяти, не сохраняя на диск.
    Возвращает изображение в виде numpy-массива.
    """
    frequencies, times, Zxx = signal.stft(
        chunk_data, 
        fs=fs, 
        nperseg=NPERSEG, 
        noverlap=NOVERLAP, 
        boundary=None
    )
    Zxx_abs = np.abs(Zxx)
    min_val = np.min(Zxx_abs)
    max_val = np.max(Zxx_abs)
    if max_val > min_val:
        Zxx_normalized = (Zxx_abs - min_val) / (max_val - min_val)
    else:
        Zxx_normalized = np.zeros_like(Zxx_abs)
    height, width = IMG_HEIGHT, IMG_WIDTH # Zxx_normalized.shape
    dpi = DPI
    fig_size = (width / dpi, height / dpi)
    fig = plt.figure(figsize=fig_size, dpi=dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(Zxx_normalized, cmap='gray', origin='lower', aspect='auto')
    fig.canvas.draw()
    buf = fig.canvas.buffer_rgba()
    img_array = np.frombuffer(buf, dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
    img_array_rgb = img_array[:, :, :3]
    plt.close(fig)
    return img_array_rgb

In [4]:
def predict_vibration_state(csv_path, model_path):
    """
    Загружает CSV файл, обрабатывает его и делает предсказание с помощью модели.
    """
    print(f"Загрузка модели из файла: {model_path}")
    try:
        model = tf.keras.models.load_model(model_path)
    except Exception as e:
        print(f"Ошибка при загрузке модели: {e}")
        return
    print(f"Чтение и обработка файла: {csv_path}")
    try:
        df = pd.read_csv(csv_path)
        if 'vibration' not in df.columns:
            print("Ошибка: в CSV файле отсутствует колонка 'vibration'.")
            return
        signal_data = df['vibration'].values
    except Exception as e:
        print(f"Ошибка при чтении CSV файла: {e}")
        return
    predictions = []
    for i in range(0, len(signal_data) - WINDOW_SIZE + 1, STEP):
        chunk = signal_data[i : i + WINDOW_SIZE]
        spectrogram_img = create_spectrogram_from_chunk_normalized(chunk, SAMPLING_RATE)
        img_tensor = tf.expand_dims(spectrogram_img, 0)
        prediction = model.predict(img_tensor, verbose=0)
        score = tf.nn.sigmoid(prediction[0][0])
        predicted_class_index = 1 if score > 0.5 else 0
        predictions.append(CLASS_NAMES[predicted_class_index])
    if not predictions:
        print("Не удалось создать ни одного фрагмента для анализа. Файл слишком короткий.")
        return
    print(f"\nАнализ завершен. Всего проанализировано фрагментов: {len(predictions)}")
    defect_count = predictions.count('defect')
    normal_count = predictions.count('normal')
    print(f"  - Фрагментов 'normal': {normal_count}")
    print(f"  - Фрагментов 'defect': {defect_count}")
    if defect_count > 0:
        final_verdict = "DEFECT"
        print("Результат: Defect")
    else:
        final_verdict = "NORMAL"
        print("Результат: Norm")
    return final_verdict

In [ ]:
INPUT_DIR = "new_data"
PATH_TO_MODEL = 'model.keras'

for file in Path(INPUT_DIR).iterdir():
    if file.is_file():
        print(f"Prediction file {file}: ")
        predict_vibration_state(file, PATH_TO_MODEL)

Prediction file new_data/87842-722749.csv: 
Загрузка модели из файла: model.keras


2025-07-14 11:18:38.252861: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-07-14 11:18:38.252896: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: maybe-pc
2025-07-14 11:18:38.252902: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: maybe-pc
2025-07-14 11:18:38.253064: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.144.3
2025-07-14 11:18:38.253085: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.144.3
2025-07-14 11:18:38.253089: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 550.144.3
2025-07-14 11:18:38.253741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions 

Чтение и обработка файла: new_data/87842-722749.csv

Анализ завершен. Всего проанализировано фрагментов: 309
  - Фрагментов 'normal': 309
  - Фрагментов 'defect': 0
Результат: Norm
Prediction file new_data/7013929-7229016.csv: 
Загрузка модели из файла: model.keras
Чтение и обработка файла: new_data/7013929-7229016.csv

Анализ завершен. Всего проанализировано фрагментов: 104
  - Фрагментов 'normal': 104
  - Фрагментов 'defect': 0
Результат: Norm
Prediction file new_data/6711221-6994406.csv: 
Загрузка модели из файла: model.keras
Чтение и обработка файла: new_data/6711221-6994406.csv

Анализ завершен. Всего проанализировано фрагментов: 137
  - Фрагментов 'normal': 137
  - Фрагментов 'defect': 0
Результат: Norm
Prediction file new_data/11804254-11931301.csv: 
Загрузка модели из файла: model.keras
Чтение и обработка файла: new_data/11804254-11931301.csv

Анализ завершен. Всего проанализировано фрагментов: 61
  - Фрагментов 'normal': 61
  - Фрагментов 'defect': 0
Результат: Norm
Prediction

In [ ]:
PATH_TO_NEW_CSV = 'csv_newsapsan_data\normal\87842-722749.csv'
PATH_TO_MODEL = 'vibration_classifier_model_1024_16k_NORM_e5.keras'

if not os.path.exists(PATH_TO_NEW_CSV):
    print(f"Файл не найден: {PATH_TO_NEW_CSV}. Укажите правильный путь.")
elif not os.path.exists(PATH_TO_MODEL):
    print(f"Модель не найдена: {PATH_TO_MODEL}. Укажите правильный путь.")
else:
    predict_vibration_state(PATH_TO_NEW_CSV, PATH_TO_MODEL)

Загрузка модели из файла: vibration_classifier_model_1024_16k_NORM_e5.keras
Чтение и обработка файла: csv_cwru_data/defect/B007_0.csv

Анализ завершен. Всего проанализировано фрагментов: 13
  - Фрагментов 'normal': 0
  - Фрагментов 'defect': 13
Результат: Defect
